# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard


In [127]:
import sqlite3
import nltk
import random
import numpy as np
import re
from collections import Counter, defaultdict

from string import punctuation

# Feel free to include your text patterns functions
# from text_functions_solutions import clean_tokenize, get_patterns
from pathlib import Path
from nltk import sent_tokenize
from nltk.corpus import stopwords

In [128]:
# set data location
data_location = Path("../datasets")

# constants
PUNCT_SET = set(punctuation)
SW_ENG = stopwords.words("english")
WHITESPACE_RE = re.compile(r"\s+")


# helper functions
def tokenize_on_ws(text):
    return WHITESPACE_RE.split(text)


def remove_punct(tokens, punct_set=PUNCT_SET):
    cleaned = []
    for token in tokens:
        tok = "".join([ch for ch in token if ch not in punct_set])
        if tok:
            cleaned.append(tok)
    return cleaned


def is_alpha(tokens):
    return [token for token in tokens if token.isalpha()]


def remove_stopwords(tokens, sw=SW_ENG):
    tokens = [token for token in tokens if token.lower() not in sw]
    return tokens


def lowercase(tokens):
    return [token.lower() for token in tokens]


def join_to_string(tokens):
    return " ".join(tokens)


def run_pipeline(text, pipeline):
    tokens = str(text)

    for transform in pipeline:
        tokens = transform(tokens)

    return tokens

In [129]:
convention_db = sqlite3.connect(data_location / "2020_Conventions.db")
convention_cur = convention_db.cursor()

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text
for each party and prepare it for use in Naive Bayes.


In [130]:
convention_data = []

# fill the above list up with items that are themselves lists. The
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party.

query_results = convention_cur.execute(
    # -- your query here, pull only 2020 data.
    # -- Remove the party "Other".
    """    
    SELECT text, party 
    FROM conventions 
    WHERE party != 'Other';
    """
)

for row in query_results:
    # store the results in convention_data
    convention_data.append([row[0], row[1]])

In [131]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

Let's look at some random entries and see if they look right.


In [132]:
random.choices(convention_data, k=5)

[['An oath.', 'Republican'],
 ['If we do not get meaningful legislation out of this Congress, we will march through the South, through the streets of Jackson, through the streets of Danville, through the streets of Cambridge, through the streets of Birmingham. James M. Lawson Jr.: ( 40:34 )  I think he is the singular figure that has tried to carry out the work of our nonviolent campaigns into the halls of Congress.',
  'Democratic'],
 ['Well, I may be kidding myself, but I think the people are ready. I think people are ready, we just got to keep pushing. We can’t let up. But thank you for what you’re doing. I really appreciate it. Thanks for joining me.',
  'Democratic'],
 ['Questions about money he made from foreign business dealings while his father was vice president.',
  'Republican'],
 ['Is to do nothing substantive to reduce it. To release prisoners as many, and as soon as possible, and to go to war with the police, the only group with the capability to protect your citizens. It

It'll be useful for us to have a large sample size than 2020 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html).


In [133]:
conv_sent_data = []

nltk.download("punkt")
nltk.download("punkt_tab")
for speech, party in convention_data:
    sentences = sent_tokenize(speech)
    conv_sent_data.append([sentences[0], party])

[nltk_data] Downloading package punkt to /home/junc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/junc/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Again, let's look at some random entries.


In [134]:
random.choices(conv_sent_data, k=5)

[['Yes, they are?', 'Republican'],
 ['California, home to our next Vice President Kamala Harris, casts 231 votes for Bernie Sanders and 263 votes for our next President, Joe Biden.',
  'Democratic'],
 ['We simply cannot endure a Biden-induced recession.', 'Republican'],
 ['I’m Sean Parnell, and it is an honor to be here.', 'Republican'],
 ['If I apply this test to Joe Biden, I can’t say yes to any of these three questions.',
  'Republican']]

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps:

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [135]:
speech_pipeline = [
    tokenize_on_ws,
    remove_punct,
    is_alpha,
    remove_stopwords,
    lowercase,
    join_to_string,
]

clean_conv_sent_data = []  # list of tuples (sentence, party), with sentence cleaned

for idx, sent_party in enumerate(conv_sent_data):
    # pass  # your code here
    # print(idx, sent_party[0], "\n")
    cleaned = run_pipeline(sent_party[0], speech_pipeline)
    # print(idx, cleaned)
    if cleaned:
        clean_conv_sent_data.append((cleaned, sent_party[1]))

random.choices(clean_conv_sent_data, k=5)

[('middle east iran threatened president approved strike killed iranian terrorist qasem soleimani',
  'Republican'),
 ('welcome', 'Democratic'),
 ('proudly displayed', 'Republican'),
 ('lot democrats support president disgusted old party old party become',
  'Republican'),
 ('yes', 'Democratic')]

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5.


In [136]:
word_cutoff = 5

tokens = [w for t, p in clean_conv_sent_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items():
    if count > word_cutoff:
        feature_words.add(word)

print(
    f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model."
)

With a word cutoff of 5, we have 628 as features in the model.


In [ ]:
def conv_features(text, fw):
    """Given some text, this returns a dictionary holding the
    feature words.

    Args:
         * text: a piece of text in a continuous string. Assumes
         text has been cleaned and case folded.
         * fw: the *feature words* that we're considering. A word
         in `text` must be in fw in order to be returned. This
         prevents us from considering very rarely occurring words.

    Returns:
         A dictionary with the words in `text` that appear in `fw`.
         Words are only counted once.
         If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
         then this would return a dictionary of
         {'quick' : True,
          'fox' :    True}

    """

    # Your code here

    ret_dict = dict()

    return ret_dict

In [ ]:
assert len(feature_words) > 0
assert conv_features("obama was the president", feature_words) == {
    "obama": True,
    "president": True,
}
assert conv_features("some people in america are citizens", feature_words) == {
    "people": True,
    "america": True,
    "citizens": True,
}

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory.


In [ ]:
featuresets = [
    (conv_features(text, feature_words), party) for (text, party) in convention_data
]

In [ ]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [ ]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

In [ ]:
classifier.show_most_informative_features(25)

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

_Your observations to come._


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and
is unindexed, so the query takes a minute or two to run on my machine.


In [ ]:
cong_db = sqlite3.connect("congressional_twitter_data.db")
cong_cur = cong_db.cursor()

In [ ]:
results = cong_cur.execute(
    """
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        """
)

results = list(results)  # Just to store it, since the query is time consuming

In [ ]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...


In [ ]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data, k=10)

In [ ]:
for tweet, party in tweet_data_sample:
    estimated_party = "??"  # you need to fill this in.
    # Fill in the right-hand side above with code that estimates the actual party

    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Now that we've looked at it some, let's score a bunch and see how we're doing.


In [ ]:
# dictionary of counts by actual party and estimated party.
# first key is actual, second is estimated
parties = ["Republican", "Democratic"]
results = defaultdict(lambda: defaultdict(int))

for p in parties:
    for p1 in parties:
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data):
    tweet, party = tp
    # Now do the same thing as above, but we store the results rather
    # than printing them.

    # get the estimated party
    estimated_party = "Gotta fill this in"

    results[party][estimated_party] += 1

    if idx > num_to_score:
        break

In [ ]:
results

### Reflections

_Write a little about what you see in the results_
